In [2]:
"""
megafloods
"""
import pandas as pd
import numpy as np
from scipy.stats import genextreme
from datetime import datetime
from datetime import timedelta

In [ ]:
# 读入数据
# Read CSV files into DataFrames
stations = pd.read_csv("./gauges.csv")
flood_data = pd.read_csv("./floods.csv")
print(stations)
print(flood_data)

In [18]:
# 有问题
import pandas as pd
import numpy as np
from datetime import datetime
from scipy.stats import genextreme
from scipy.stats import percentileofscore

def find_mf_func(stations_tab, amax_tab, start_year):
    megafloods_df = pd.DataFrame(columns=["id", "LAT", "LON", "area", "Q_MF", "year_MF", "date_MF"])
    
    nn = 0  # count number of outliers
    for i in range(stations_tab.shape[0]):
        dummy = amax_tab[amax_tab["id"] == stations_tab.loc[i, "id"]]
        dummy = dummy[~np.isnan(dummy["q"])]
        if len(dummy["q"]) < 10:
            continue  # exclude short series
        
        # megafloods criteria:
        
        # 1. outlier
        dummy_quant = np.percentile(dummy["q"], q=[25, 75])
        dummy_upper = dummy_quant[1] + 3 * (dummy_quant[1] - dummy_quant[0])
        index_outlier = np.where(dummy["q"] >= dummy_upper)[0]
        
        if len(index_outlier) == 0:
            continue
        
        index_MF = []
        for tt in index_outlier:
            ty = dummy.loc[tt, "year"]
            nyears_before = len(dummy["q"][dummy["year"] < ty])
            
            # 3. after start_year and with at least 20 years of data before event
            if nyears_before < 20:
                continue
            if ty < start_year:
                continue
            
            # 2. surprising (Tl/Tsl>3) record-breaking event
            qmax_old = dummy["q"][dummy["year"] <= ty]
            shape, loc, scale = genextreme.fit(qmax_old)
            Tl = np.max(qmax_old)
            Tsl = genextreme.ppf(1 - 1 / len(qmax_old), shape, loc, scale)
            rRP = Tl / Tsl
            if rRP > 3:
                index_MF.append(tt)
        
        index_outlier = index_MF
        
        if len(index_outlier) > 0:
            for oo in index_outlier:
                nn += 1
                megafloods_df.loc[nn, "id"] = stations_tab.loc[i, "id"]
                megafloods_df.loc[nn, "LAT"] = stations_tab.loc[i, "lat"]
                megafloods_df.loc[nn, "LON"] = stations_tab.loc[i, "lon"]
                megafloods_df.loc[nn, "area"] = float(stations_tab.loc[i, "area"])
                megafloods_df.loc[nn, "Q_MF"] = dummy.loc[oo, "q"]  # outlier discharge
                megafloods_df.loc[nn, "year_MF"] = dummy.loc[oo, "year"]  # outlier year
                megafloods_df.loc[nn, "date_MF"] = datetime.strptime(
                    f"{dummy.loc[oo, 'year']}-{dummy.loc[oo, 'month']}-{dummy.loc[oo, 'day']}",
                    "%Y-%m-%d"
                )  # outlier date
    
    return megafloods_df


# 完成数据集的建立
stations = pd.read_csv("./gauges.csv")
flood_data = pd.read_csv("./floods.csv")
print(find_mf_func(stations,flood_data,2000))

KeyError: 2

第一部分：筛选数据
问题：统计计算的方法大概率是有问题的，可能需要自己写
第二部分：各类计算
问题：待定
第三部分：预测

In [1]:
import pandas as pd
from scipy.stats import genextreme
from scipy import interpolate
import numpy as np

def find_mf_func(stations_tab, amax_tab, start_year):
    megafloods_df = pd.DataFrame(columns=['id', 'LAT', 'LON', 'area', 'Q_MF', 'year_MF', 'date_MF'])
  
    nn = 0  # count number of outliers
    for i in range(len(stations_tab)):
        station_id = stations_tab['id'][i]
        dummy = amax_tab[amax_tab['id'] == station_id].dropna(subset=['q'])
        # dummy = amax_tab[amax_tab['id'] == station_id]
        
        if len(dummy) < 10:
            continue  # exclude short series
        
        # megafloods criteria:
        
        # 1. outlier 
        
        # dummy_quant = np.percentile(dummy['q'],[25,75],method='')
        dummy_quant = np.percentile(dummy['q'],[25,75],method="linear")   # 我去，这个值有点意思 --- 可以细细琢磨下
        # dummy_quant = np.percentile(dummy['q'],q = [25,75],method='linear')
        dummy_upper = dummy_quant[1] + 3 * (dummy_quant[1] - dummy_quant[0])
        # dummy_quant = dummy['q'].quantile([0.25, 0.75])
        # dummy_upper = float(dummy_quant[0.75]) + 3 * float((dummy_quant[0.75]) - float(dummy_quant[0.25]))
        index_outlier = dummy[dummy['q'] >= dummy_upper].index
        if len(index_outlier) == 0:
            continue
        
        index_MF = []
        for tt in index_outlier:
            ty = dummy['year'][tt]
            nyears_before = len(dummy[dummy['year'] < ty])
            
            # 3. after start.year and with at least 20 years of data before event
            if nyears_before < 20 or ty < start_year:
                continue
            
            # 2. surprising (Tl/Tsl > 3) record-breaking event
            qmax_old = dummy[dummy['year'] <= ty]['q']
            local_gev = genextreme.fit(qmax_old)
            neP = genextreme.cdf(qmax_old, *local_gev)
            RP = 1 / (1 - neP)
            RP[RP == float('inf')] = 10 ** 5
            rRP = RP[-1] / max(RP[:-1])
            if rRP > 3:
                index_MF.append(tt)
                
        index_outlier = index_MF
        if len(index_MF) > 0:
            for oo in index_MF:
                nn += 1
                try:
                    megafloods_df.loc[nn] = [stations_tab['id'][i], stations_tab['lat'][i], stations_tab['lon'][i],
                                         float(stations_tab['area'][i]), dummy['q'][oo], dummy['year'][oo],
                                         pd.to_datetime(
                                             f"{int(dummy['year'][oo])}-{int(dummy['month'][oo])}-{int(dummy['day'][oo])}")]
                except ValueError:
                    megafloods_df.loc[nn] = [stations_tab['id'][i], stations_tab['lat'][i], stations_tab['lon'][i],
                                         float(stations_tab['area'][i]), dummy['q'][oo], dummy['year'][oo],-1]
    
    return megafloods_df

stations = pd.read_csv("./gauges.csv")
flood_data = pd.read_csv("./floods.csv")
megafloods_df = find_mf_func(stations,flood_data,2000)
# print(find_mf_func(stations,flood_data,2000))


In [ ]:
# 第二部分
import pandas as pd

stations['mean_q'] = pd.Series([None] * len(stations))
stations['max_q'] = pd.Series([None] * len(stations))

for i in range(len(stations)):
    qq = flood_data[flood_data['id'] == stations['id'][i]][2]
    stations.at[i, 'mean_q'] = round(qq.mean(), 5)
    stations.at[i, 'max_q'] = qq.max()
    # del qq


In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import mstats

def envelope_curve(stations_tab, amax_tab):
    ENV_tab = pd.DataFrame(columns=["slope_qr_median", "intercept_qr_median", "slope_qr_env", "intercept_qr_env", "intercept_env", "n_stat_years"])
    
    for rr in range(1, 6):
        print(rr)
        dummy_stations = stations_tab[stations_tab["regions1"] == rr]
        dummy_q = amax_tab[amax_tab["id"].isin(dummy_stations["id"]) & (amax_tab["q"] > 0)]
        dummy_q["qlog10"] = np.log10(dummy_q["q"])
        dummy_q["Alog10"] = np.nan
        
        for i in range(len(dummy_stations)):
            ind_stat = dummy_q.index[dummy_q["id"] == dummy_stations["id"][i]]
            dummy_q.at[ind_stat, "Alog10"] = np.log10(dummy_stations["area"][i])
        
        # quantile regression z=0.5
        p = 1
        dummy_rq = mstats.quantreg(dummy_q["qlog10"], dummy_q["Alog10"], alpha=0.5)
        ENV_tab[ "slope_qr_median"][rr-1] = dummy_rq.params[1] # slope of qr
        ENV_tab["intercept_qr_median"][rr-1] = dummy_rq.params[0] # intercept of qr
        
        # quantile regression z=0.999
        dummy_rq = mstats.quantreg(dummy_q["qlog10"], dummy_q["Alog10"], alpha=0.999)
        ENV_tab[ "slope_qr_env"][rr-1]  = dummy_rq.params[1] # slope of qr
        ENV_tab["intercept_qr_env"][rr-1]  = dummy_rq.params[0] # intercept of qr
        
        intercept = -ENV_tab["slope_qr_env"][rr-1] * np.log10(dummy_stations["area"]) + np.log10(dummy_stations["max_q"])
        ENV_tab["intercept_env"][rr-1]  = max(intercept) # intercept of envelope
        ENV_tab["n_stat_years"][rr-1]  = len(dummy_q) # nstatyears
    
    return ENV_tab

ENV_reg = envelope_curve(stations,flood_data)

In [ ]:
import pandas as pd
# EDT 1
def ext_data_tab1_func(envelope_tab):
    ext_data_table1 = pd.DataFrame(data=None, columns=["Slope_Envelope", "Intercept_Envelope", "Intercept_Envelope_1000km2", "Slope_median_regression", "Intercept_median_regression"], index=envelope_tab.index)
    ext_data_table1["Slope_Envelope"] = envelope_tab["slope_qr_env"]
    ext_data_table1["Intercept_Envelope"] = envelope_tab["intercept_env"]
    ext_data_table1["Intercept_Envelope_1000km2"] = 10**(envelope_tab["intercept_env"] + envelope_tab["slope_qr_env"] * (np.log10(1000)))
    ext_data_table1["Slope_median_regression"] = envelope_tab["slope_qr_median"]
    ext_data_table1["Intercept_median_regression"] = envelope_tab["intercept_qr_median"]
    ext_data_table1 = round(ext_data_table1, 2)
    return ext_data_table1

ExtDataTable1 = ext_data_tab1_func(ENV_reg)


In [ ]:
import numpy as np
import pandas as pd
# EDT 2
ExtDataTable2 = pd.DataFrame(columns=["Number_of_gauges", "Avg_Ratio_max_mean_Q", "Number_of_targets", "Percentage_of_targets", "Avg_Ratio_max_mean_Q_targets"])

for rr in range(1, 6):
    dummy_stations = stations[stations["regions1"] == rr]
    ExtDataTable2.at[rr-1, "Number_of_gauges"] = len(dummy_stations)
    ExtDataTable2.at[rr-1, "Avg_Ratio_max_mean_Q"] = round(np.mean(dummy_stations["max_q"] / dummy_stations["mean_q"]), 2)
    dummy_mf = dummy_stations[dummy_stations["id"].isin(megafloods_df["id"])]
    ExtDataTable2.at[rr-1, "Number_of_targets"] = len(dummy_mf)
    ExtDataTable2.at[rr-1, "Percentage_of_targets"] = round(ExtDataTable2.at[rr-1, "Number_of_targets"] * 100 / ExtDataTable2.at[rr-1, "Number_of_gauges"], 1)
    ExtDataTable2.at[rr-1, "Avg_Ratio_max_mean_Q_targets"] = round(np.mean(dummy_mf["max_q"] / dummy_mf["mean_q"]), 2)

ExtDataTable2.index = ExtDataTable2.index + 1


In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import iqr
from sklearn.linear_model import LinearRegression
# Fig. 2 & Fig. 4

# mean and cv up to target years
def mhq_cv_func(stations_tab, amax_tab, target_years):
    EU_MHQ_CV_list = {}
    for YY in range(len(target_years)):
        print(target_years[YY])
        TARGET_YEAR = target_years[YY]
        EU_MHQ_CV = pd.DataFrame(np.nan, index=range(stations_tab.shape[0]), columns=["id", "lat", "lon", "area", "MHQ", "CV", "MHQ_100"])
        EU_MHQ_CV[["id", "lat", "lon", "area"]] = stations_tab[["id", "lat", "lon", "area"]]
        for rrr in range(EU_MHQ_CV.shape[0]):
            dummy_tab = amax_tab[(amax_tab["id"]==EU_MHQ_CV.iloc[rrr, 0])&(amax_tab["year"]<TARGET_YEAR)].dropna(subset=["q"])
            if dummy_tab.shape[0] < 10:
                continue
            dummy_Q = dummy_tab["q"].values
            dummy_Y = dummy_tab["year"].values
            dummy_quant = np.quantile(dummy_Q, q=[0.25, 0.75])
            dummy_upper = dummy_quant[1] + 3*(dummy_quant[1] - dummy_quant[0])
            dummy_Y = dummy_Y[dummy_Q <= dummy_upper]
            dummy_Q = dummy_Q[dummy_Q <= dummy_upper]
            EU_MHQ_CV.iloc[rrr, 4] = np.mean(dummy_Q)
            EU_MHQ_CV.iloc[rrr, 5] = iqr(dummy_Q)/EU_MHQ_CV.iloc[rrr, 4]
        # normalize MHQ to 100km2
        area_regr = LinearRegression().fit(np.log10(EU_MHQ_CV[["area"]]), np.log10(EU_MHQ_CV[["MHQ"]]))
        EU_MHQ_CV.iloc[:, 6] = 10**(np.log10(EU_MHQ_CV["MHQ"]) - area_regr.coef_[0]*np.log10(EU_MHQ_CV["area"]) + area_regr.coef_[0]*np.log10(100))
        EU_MHQ_CV_list[target_years[YY]] = EU_MHQ_CV
    return EU_MHQ_CV_list

EU_MHQ_CV_list = mhq_cv_func(stations, flood_data, list(range(2000, 2023)))


In [ ]:
#第三部分
import numpy as np
import pandas as pd

def predict_mf_function(mf_list, stations_tab, target_years, amax_tab, mhq_cv_tab, env_tab):
    # specify weights and distance limit for finding donor group
    w1 = 1
    w2 = 1
    w3 = 1
    DIST = 1

    # prepare lists for saving results
    REG_MEGAFLOODS_list = [None] * 5  # List megafloods per each region
    REG_ENVELOPE_param = [None] * 5  # Envelope parameters
    REG_DONORS_list = [None] * 5  # Donor list per region per megaflood

    for REG in range(1, 6):  # loop on Regions
        REG_list_stations = stations_tab['id'][stations_tab['regions1'] == REG]
        MEGAFLOODS_list = mf_list[mf_list['id'].isin(REG_list_stations)]
        DONORS_list = [None] * MEGAFLOODS_list.shape[0]
        ENVELOPE_param = pd.DataFrame(np.nan, index=range(MEGAFLOODS_list.shape[0]), columns=["slope", "intercept.region", "intercept.mf", "q.predicted", "n_statyears_donors", "n_stat_donors"])

        for i in range(MEGAFLOODS_list.shape[0]):  # loop on megafloods
            print(i)

            TARGET_YEAR = MEGAFLOODS_list['year_MF'].iloc[i]
            TARGET_Q = MEGAFLOODS_list['Q_MF'].iloc[i]
            TARGET_DATE = MEGAFLOODS_list['date_MF'].iloc[i]
            code_mf = MEGAFLOODS_list['id'].iloc[i]
            EU_MHQ_CV = mhq_cv_tab[target_years.index(TARGET_YEAR)]
            EU_MHQ_CV = EU_MHQ_CV.dropna(subset=['MHQ'])
            EU_MHQ_CV = EU_MHQ_CV[EU_MHQ_CV['id'].isin(REG_list_stations)]

            # distances
            ind_stat = EU_MHQ_CV[EU_MHQ_CV['id'] == code_mf].index
            sd1 = (np.log10(EU_MHQ_CV['area']) - np.mean(np.log10(EU_MHQ_CV['area']))) / np.std(np.log10(EU_MHQ_CV['area']))
            sd2 = (np.log10(EU_MHQ_CV['MHQ_100']) - np.mean(np.log10(EU_MHQ_CV['MHQ_100']))) / np.std(np.log10(EU_MHQ_CV['MHQ_100']))
            sd3 = (np.log10(EU_MHQ_CV['CV']) - np.mean(np.log10(EU_MHQ_CV['CV']))) / np.std(np.log10(EU_MHQ_CV['CV']))
            d1 = sd1 - sd1[ind_stat].values[0]
            d2 = sd2 - sd2[ind_stat].values[0]
            d3 = sd3 - sd3[ind_stat].values[0]
            dd = np.sqrt((d1**2) * w1 + (d2**2) * w2 + (d3**2) * w3)
            # donor group
            group = EU_MHQ_CV.loc[dd <= DIST, 'id']
            stations_Reg_group = stations_tab[stations_tab['id'].isin(group)]
            EU_MHQ_CV_group = EU_MHQ_CV[EU_MHQ_CV['id'].isin(group)]
            # characteristics of donor catchments
            DONORS = pd.DataFrame(columns=["id", "lat", "lon", "area", "SFOR.B", "SFOR.year.B", "SFOR.date.B"])
            for s in range(EU_MHQ_CV_group.shape[0]):
                DONORS.loc[s, "id"] = int(EU_MHQ_CV_group.iloc[s]['id'])
                DONORS.loc[s, "lat"] = float(EU_MHQ_CV_group.iloc[s]['lat'])
                DONORS.loc[s, "lon"] = float(EU_MHQ_CV_group.iloc[s]['lon'])
                DONORS.loc[s, "area"] = float(EU_MHQ_CV_group.iloc[s]['area'])
                dummy = amax_tab[amax_tab['id'] == EU_MHQ_CV_group.iloc[s]['id']]
                QQ = dummy[dummy['year'] < TARGET_YEAR]['q'].values
                DD = pd.to_datetime(dummy[dummy['year'] < TARGET_YEAR][['year', 'month', 'day']].astype(str).agg('-'.join, axis=1))
                YY = dummy[dummy['year'] < TARGET_YEAR]['year'].values
                ind_SFOR = np.argmax(QQ)
                if len(ind_SFOR) > 1:
                    ind_SFOR = np.max(ind_SFOR)
                DONORS.loc[s, "SFOR.B"] = QQ[ind_SFOR]
                DONORS.loc[s, "SFOR.year.B"] = YY[ind_SFOR]
                DONORS.loc[s, "SFOR.date.B"] = DD[ind_SFOR]
                dummy, ind_SFOR, QQ, YY, DD = None, None, None, None, None

            # envelope with region slope
            ENVELOPE_param.loc[i, "slope"] = env_tab['slope_qr_env'][REG - 1]
            ENVELOPE_param.loc[i, "intercept.region"] = env_tab['intercept_qr_env'][REG - 1]
            intercept = -ENVELOPE_param.loc[i, "slope"] * np.log10(DONORS['area']) + np.log10(DONORS['SFOR.B'])
            DONORS['intercept'] = intercept
            DONORS = DONORS.sort_values(by="intercept", ascending=False)
            DONORS_list[i] = DONORS
            ENVELOPE_param.loc[i, "intercept.mf"] = np.max(intercept)
            ENVELOPE_param.loc[i, "q.predicted"] = ENVELOPE_param.loc[i, "intercept.mf"] + ENVELOPE_param.loc[i, "slope"] * np.log10(MEGAFLOODS_list['area'].iloc[i])
            ENVELOPE_param.loc[i, "n_statyears_donors"] = amax_tab[amax_tab['id'].isin(DONORS['id'])].shape[0]  # n station-years in donor group
            ENVELOPE_param.loc[i, "n_stat_donors"] = len(group)

        REG_MEGAFLOODS_list[REG - 1] = MEGAFLOODS_list
        REG_ENVELOPE_param[REG - 1] = ENVELOPE_param
        REG_DONORS_list[REG - 1] = DONORS_list

    REG_MEGAFLOODS_list = dict(zip(range(1, 6), REG_MEGAFLOODS_list))
    REG_ENVELOPE_param = dict(zip(range(1, 6), REG_ENVELOPE_param))
    REG_DONORS_list = dict(zip(range(1, 6), REG_DONORS_list))

    RES = {"mf.list": REG_MEGAFLOODS_list, "EC.param": REG_ENVELOPE_param, "donor.list": REG_DONORS_list}
    return RES

MF_predicted = predict_mf_function(megafloods_df, stations, list(range(2000, 2023)), flood_data, EU_MHQ_CV_list, ENV_reg)


In [ ]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from math import atan2, pi

def PREDvsOBS_plot(mf_pred_list):
    # prepare lists for saving results
    id_mf = []
    lat = []
    lon = []
    region = []
    q_obs = []
    q_pred = []
    ratio = []
    doy_target = []
    doy_donors = []
    r_donors = []

    for rr in range(1, 6):
        ENVELOPE_param = mf_pred_list["EC.param"][rr-1]
        MEGAFLOODS_list = mf_pred_list["mf.list"][rr-1]
        DONORS_list = mf_pred_list["donor.list"][rr-1]

        id_mf_r = MEGAFLOODS_list['id'].tolist()
        lat_r = MEGAFLOODS_list['LAT'].tolist()
        lon_r = MEGAFLOODS_list['LON'].tolist()
        region_r = [rr] * len(id_mf_r)

        # q
        q_obs_r = MEGAFLOODS_list['Q_MF'].tolist()
        q_pred_r = np.round(10**ENVELOPE_param['q.predicted'], 5).tolist()

        # timing
        m = 365.25
        doy_target_r = [(datetime.strptime(date_str, '%Y-%m-%d').timetuple().tm_yday) * 2 * pi / m for date_str in MEGAFLOODS_list['date_MF']]
        teta_donors_r = []
        doy_donors_r = []
        r_donors_r = []
        for mm in range(len(DONORS_list)):
            # 10 donors
            DONORS = DONORS_list[mm]
            uu = min(10, DONORS.shape[0])
            DONORS = DONORS.iloc[:uu, :]
            doy = [(datetime.strptime(date_str, '%Y-%m-%d').timetuple().tm_yday) * 2 * pi / m for date_str in DONORS['SFOR.date.B']]
            x = [np.cos(t) for t in doy]
            y = [np.sin(t) for t in doy]
            x_donor = np.mean(x)
            y_donor = np.mean(y)
            teta_donors_r.append(atan2(y_donor, x_donor))
            doy_donors_r.append(round(teta_donors_r[mm]*m/(2*pi), 0))
            if doy_donors_r[mm] < 0:
                doy_donors_r[mm] += m
            r_donors_r.append(round((x_donor**2 + y_donor**2)**0.5, 2))

        # append to lists
        id_mf.extend(id_mf_r)
        lat.extend(lat_r)
        lon.extend(lon_r)
        region.extend(region_r)
        q_obs.extend(q_obs_r)
        q_pred.extend(q_pred_r)
        doy_target.extend(doy_target_r)
        doy_donors.extend(doy_donors_r)
        r_donors.extend(r_donors_r)

    ratio = np.round(np.array(q_obs) / np.array(q_pred), 2).tolist()
    PREDvsOBS_mf = pd.DataFrame({
        'id.mf': id_mf,
        'lat': lat,
        'lon': lon,
        'region': region,
        'Q.obs': q_obs,
        'Q.pred': q_pred,
        'Ratio': ratio,
        'doy.target': doy_target,
        'doy.donors': doy_donors,
        'R.donors': r_donors
    })
    return PREDvsOBS_mf

PREDvsOBS_mf = PREDvsOBS_plot(MF_predicted)